## Requirements 

In [1]:
# %pip install riskparityportfolio
# %pip install seaborn
# %pip install Riskfolio-Lib
# %pip install OptimalCluster

## Import libraries

In [2]:
import math
import pandas as pd
import numpy as np

import os
import csv

import datetime
import json
from typing import List, Tuple

import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

In [3]:
from StocksReader import *
from Stocks import *
from Hoeffding import *
from HCBAA import *
from CCD import *
from ComparisonMeasures import *

(CVXPY) Jun 11 02:05:40 PM: Encountered unexpected exception importing solver CPLEX:
ImportError("dlopen(/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cplex/_internal/py38_cplex2010.so, 0x0002): tried: '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cplex/_internal/py38_cplex2010.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cplex/_internal/py38_cplex2010.so' (no such file), '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cplex/_internal/py38_cplex2010.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64'))")
(CVXPY) Jun 11 02:05:40 PM: Encountered unexpected exception importing solver CPLEX:
ImportError("dlopen(/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/

## Experiments

In [4]:
def get_profitability_portfolio_one_day(weights, stocks_perfomance):
    if len(stocks_perfomance[0].dates) == 7:
        N = 7
    else:
        N = len(stocks_perfomance[0].dates)
    profitability_portfolio = []
    for day in np.arange(N):
        prof = 0
        for stock in stocks_perfomance:
            prof += stock.returns.iloc[day]*weights[stock.id_]
        profitability_portfolio.append(prof)
    return sum(profitability_portfolio)

def get_prof(resulting_portfolios, stocks_perfomance):
    returns = {}
    for portfolio in resulting_portfolios:
        returns[portfolio] = get_profitability_portfolio_one_day(resulting_portfolios[portfolio], stocks_perfomance)
    return returns

def get_risk(weights, covariance_matrix):
    risks = {}
    for name_portfolio in weights:
        risks[name_portfolio] = math.sqrt(np.transpose(weights[name_portfolio]).dot(covariance_matrix).dot(weights[name_portfolio]))
    return risks

In [ ]:
window_size = 252

start_date = "2020-01-01"
end_date =   '2023-12-31'

switch = 10

single, complete, average, ward, DBHT  = [], [], [], [], []
MV, MDP, ERC, IVRB  = [], [], [], []
EW = []


ReaderData = ReaderStocksData("../Russia")
# ReaderData = ReaderStocksDatayfinance("../South Africa")

DATA_OF_STOCKS_FOR_BUILDING_MODEL, _ , TICKERS = ReaderData.load_data(str(start_date), str(end_date))
# DATA_OF_STOCKS_FOR_BUILDING_MODEL = DATA_OF_STOCKS_FOR_BUILDING_MODEL[:switch]
# TICKERS = TICKERS[:switch]

balanced_portfolio = {'balance': [1/len(DATA_OF_STOCKS_FOR_BUILDING_MODEL) for i in range(len(DATA_OF_STOCKS_FOR_BUILDING_MODEL))]}

for i in range(0, len(DATA_OF_STOCKS_FOR_BUILDING_MODEL[0].returns) - window_size - 7, 7):
    print('============================================================')
    print(i, ' - ' ,i + window_size, ' ->', i+window_size + 7)


    DATA_OF_STOCKS_FOR_BUILDING_MODEL, _ , TICKERS = ReaderData.load_data(str(start_date), str(end_date))
    # DATA_OF_STOCKS_FOR_BUILDING_MODEL = DATA_OF_STOCKS_FOR_BUILDING_MODEL[:switch]
    # TICKERS = TICKERS[:switch]

    for stock in DATA_OF_STOCKS_FOR_BUILDING_MODEL:
        stock.dates = stock.dates[i : i+window_size]
        stock.returns =  stock.returns[ i : i+window_size]
    
    portfolio_cluster = HCBAA(DATA_OF_STOCKS_FOR_BUILDING_MODEL, TICKERS)
    portfolios_on_clustering = portfolio_cluster.HCBAA_portfolios()

    portfolios_optimization = CCD(DATA_OF_STOCKS_FOR_BUILDING_MODEL, TICKERS)
    portfolios_tradition = portfolios_optimization.RiskBudgetingPortfolios()

    DATA_OF_STOCKS_FOR_BUILDING_MODEL, _ , TICKERS = ReaderData.load_data(str(start_date), str(end_date))
    # DATA_OF_STOCKS_FOR_BUILDING_MODEL = DATA_OF_STOCKS_FOR_BUILDING_MODEL[:switch]
    # TICKERS = TICKERS[:switch]

    for stock in DATA_OF_STOCKS_FOR_BUILDING_MODEL:
        stock.dates = stock.dates[i+window_size : i+window_size + 7]
        stock.returns =  stock.returns[i+window_size : i+window_size + 7]
    
    
    returns_clustering = get_prof(portfolios_on_clustering, DATA_OF_STOCKS_FOR_BUILDING_MODEL)
    risk_clustering = get_risk(portfolios_on_clustering, portfolio_cluster.covariance_matrix)

    returns_tradition = get_prof(portfolios_tradition, DATA_OF_STOCKS_FOR_BUILDING_MODEL)
    risk_tradition = get_risk(portfolios_tradition, portfolios_optimization.covariance_matrix)

    returns_balances = get_prof(balanced_portfolio, DATA_OF_STOCKS_FOR_BUILDING_MODEL)
    risk_balances = get_risk(balanced_portfolio, portfolio_cluster.covariance_matrix)

    print('======================== доходности ========================')
    print(returns_clustering)
    print(returns_tradition)
    print(returns_balances)
    print('========================== риск ===========================')
    print(risk_clustering)
    print(risk_tradition)
    print(risk_balances)

    single.append((returns_clustering['single'], portfolios_on_clustering['single'], risk_clustering['single']))
    complete.append((returns_clustering['complete'], portfolios_on_clustering['complete'], risk_clustering['complete']))
    average.append((returns_clustering['average'], portfolios_on_clustering['average'], risk_clustering['average']))
    ward.append((returns_clustering['ward'], portfolios_on_clustering['ward'], risk_clustering['ward']))
    DBHT.append((returns_clustering['DBHT'], portfolios_on_clustering['DBHT'], risk_clustering['DBHT']))

    MV.append((returns_tradition['MV'], portfolios_tradition['MV'], risk_tradition['MV']))
    MDP.append((returns_tradition['MDP'], portfolios_tradition['MDP'], risk_tradition['MDP']))
    ERC.append((returns_tradition['ERC'], portfolios_tradition['ERC'], risk_tradition['ERC']))
    IVRB.append((returns_tradition['IVRB'], portfolios_tradition['IVRB'], risk_tradition['IVRB']))

    EW.append((returns_balances['balance'], balanced_portfolio['balance'], risk_balances['balance']))
    

## Metrics

In [6]:
Comparison_Measures = ComparisonMeasures()
list_file  = [single, complete, average, ward, DBHT, MV, MDP, ERC, IVRB, EW]
list_file_name = ['single', 'complete', 'average', 'ward', 'DBHT', 'MV', 'MDP', 'ERC', 'IVRB', 'EW']

results_metric = {}
for res in range(len(list_file)):
  results_metric[list_file_name[res]] = Comparison_Measures.get_metric(list_file[res])

results_metric = pd.DataFrame(results_metric).T

In [ ]:
results_metric

## Graphs

In [8]:
returns_portfolios = {}
risks_portfolios = {}
for res in range(len(list_file)):
  returns_portfolios[list_file_name[res]] = [t[0] for t in list_file[res]]
  risks_portfolios[list_file_name[res]] = [t[2] for t in list_file[res]]

In [ ]:
plt.figure(figsize=(10, 6))

colors = plt.cm.Paired(np.linspace(0, 1, len(list(returns_portfolios.keys()))))
for (key, values), color in zip(returns_portfolios.items(), colors):
    plt.plot(values,  linestyle='-', label = key, color = color)

plt.legend()
plt.title('Доходности портфелей')
plt.xlabel('время')
plt.ylabel('доходность')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
colors = plt.cm.Paired(np.linspace(0, 1, len(list(risks_portfolios.keys()))))
for (key, values), color in zip(risks_portfolios.items(), colors):
    plt.plot(values,  linestyle='-', label = key, color = color)



plt.legend()
plt.title('Риск портфелей')
plt.xlabel('время')
plt.ylabel('риск')
plt.grid(True)
plt.show()

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(20, 10))
for i, key  in enumerate(risks_portfolios.keys()):

    row = i // 2
    col = i % 2
    

    axs[row, col].plot(returns_portfolios[key], linestyle='-', label='доходность', color='green')
    axs[row, col].plot(risks_portfolios[key], linestyle='-', label='риск', color='red')
    

    axs[row, col].legend()
    

    axs[row, col].set_title(f'График портфеля {key}')
    axs[row, col].set_xlabel('Время')
    axs[row, col].set_ylabel('Доходность/Риск')
    

    axs[row, col].grid(True)


fig.suptitle('Дохоность/Риск портфелей', fontsize=16)

plt.tight_layout()
plt.show()
